# Vibrational spectra of water

In this exercise we will compare the vibrational modes of water computed with a
static method and with molecular dynamics. We will use DFT, DFTB and PM6 methods.

## DFT method

Density functional theory can be used to calculate vibrational frequencies of molecules. In this example we will calculate the vibrational frequencies for a water molecule.
First we find an optimal structure for a single water molecule. We will use GPAW calculator for this task.

In [ ]:
from ase import *
from ase.io import read, write
from ase.optimize import QuasiNewton
from ase.structure import molecule
from gpaw import *
from IPython.display import Image
import warnings
import os
import numpy as np
import matplotlib.pyplot as plt
from ase.vibrations import Vibrations
warnings.filterwarnings('ignore')

In [ ]:
#MOLECULE VIEWER CODE
from IPython.display import HTML
def atoms_to_html(atoms):
    'Return the html representation the atoms object as string'

    from tempfile import NamedTemporaryFile

    with NamedTemporaryFile('r+', suffix='.html') as ntf:
        atoms.write(ntf.name, format='html')
        ntf.seek(0)
        html = ntf.read()
    return html

Teeme vee molekuli, optimeerime ja salvestame ta geomeetria.

In [ ]:
mol = molecule('H2O', cell=[8, 8, 8], calculator=GPAW(h=.18, mode='lcao', basis='dzp', xc='PBE'))
mol.center()
dyn = QuasiNewton(mol, trajectory='molecule.traj')
dyn.run(fmax=0.05)
write('mol.xyz', mol)

In [ ]:
HTML(atoms_to_html(mol))

Calculate the vibrational modes of a H<sub>2</sub>O molecule. Calculated frequencies will appear at the very end of the output. Compare them to literature values, which are 1595 cm<sup>-1</sup> for the bending mode, 3657 cm<sup>-1</sup> for the symmetric stretching mode and 3756 cm<sup>-1</sup> for the anti-symmetric stretching mode. How good is the accuracy and what are possible error sources?

In [ ]:
Image("water.png") #Vedela faasi IR spekter
#This IR spectrum is from the Coblentz Society's evaluated infrared reference spectra collection.

In [ ]:
# Create vibration calculator
vib = Vibrations(mol)
vib.run()
vib.summary(method='frederiksen') # ette antud 3 aatomit, igat nihutatakse mitu korda,
                                #ulejaanud sel ajal paigal. Nii otsustab, kuhu suunas liigutada

# Make trajectory files to visualize normal modes:
for mode in range(9):
    vib.write_mode(mode)

Now we want to look at the modes to see how the atoms move. For this we use the files vib.*n*.traj where *n* is the number of the mode counted in the order they are printed out. You can look at these trajectories with the ase-gui command - click Play to play the movie. Do they look like you expected and what would you have expected (you may have learned something about symmetry groups at one point)? Did you assign the modes correctly in the previous question?

In [ ]:
#from subprocess import call
#call(["ase-gui","vib.7.traj"]) # siin saab muuta numbrit, siis saab teistsuguse v]nkumise, vaja ase gui


## DFTB method

Density Functional based Tight Binding method makes computationally feasible very large calculations. In this example we will run molecular dynamics with DFTB in order to obtain the vibrational spectra of a single water molecule.

The required files and programmes can be find in the exercise directory. You will only need to execute them.

In [ ]:
DFTB = 'mpirun -np 8 cp2k.popt IR-DFTB.in > IR-DFTB.out' 
os.system(DFTB)

Dipole moment and derivatives are calculated in this simulation and saved in a file dip\*traj. From these outputs, dipole.x computes the autocorrelation function of the dipole derivative and its Fourier transform (frequency domain).

In [ ]:
dipole = './dipole.x < dipole.in'
os.system(dipole)

Let us plot the result

In [ ]:
data = np.genfromtxt("dip_dip_correlation.freq", delimiter=28, skip_header=1)

freq = [row[0] for row in data]
intv = [row[1] for row in data]

plt.plot(freq, intv)
plt.xlabel('frequency (1/cm)')
plt.xlim([4000,1000])
plt.ylabel('correlation (a.u.)')
plt.savefig("IR-DFTB-mol.png")
#Image("IR-DFTB-mol.png")

## PM6 method

Semi-empirical methods can be also used for calculating (very rough) IR spectra from Molecular Dynamics. In this example we perform a PM6 Molecular Dynamics
simulation of water, and localize the molecular orbitals to obtain Wannier centers.

In [ ]:
PM6 = 'cp2k.popt IR-PM6.in > IR-PM6.out'
os.system(PM6)

The trajectory methanol_wannier.xyz is then opened with TRAVIS. TRAVIS is an
interactive program, it asks the user questions to answer. No input file needs to be prepared before. Please take the default values pressing ENTER, expect for the following questions:

- Enter length of cell vector in pm: (Enter "2000")
- Which functions to compute (comma separated)? (Enter "ir")

To run TRAVIS, first call a terminal, then type in "./travis -p IR-wannier.xyz" and press ENTER.

In [ ]:
#from subprocess import call
#call(["gnome-terminal"])

Let us plot the result

In [ ]:
data = np.genfromtxt("spectrum_[dip_global].d1.csv", delimiter=";", skip_header=1)

freq = [row[0] for row in data]
intv = [row[1] for row in data]

plt.plot(freq, intv)
plt.xlabel('frequency (1/cm)')
plt.xlim([4000,1000])
plt.ylabel('correlation (a.u.)')
plt.savefig("test.png")

## DFTB method applied to bulk

In [ ]:
import os
DFTB = 'mpirun -np 8 cp2k.popt IR-DFTB-bulk.in > IR-DFTB-bulk.out' 
os.system(DFTB)

In [ ]:
dipole = './dipole.x < dipole_bulk.in'
os.system(dipole)

In [ ]:
data = np.genfromtxt("dip_dip_correlation.freq", delimiter=28, skip_header=1)

freq = [row[0] for row in data]
intv = [row[1] for row in data]

plt.plot(freq, intv)
plt.xlabel('frequency (1/cm)')
plt.xlim([4000,1000])
plt.ylabel('correlation (a.u.)')
plt.savefig("IR-DFTB-bulk.png")

I ran the simulations for 20 times longer, here are the results.

First, one molecule DFTB.

In [ ]:
Image("IR-DFTB-mol-long.png")

One molecule PM6.

In [ ]:
Image("test-long.png")

DFTB with 32 water molecules.

In [ ]:
Image("IR-DFTB-bulk-long.png")